In [106]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os
import math



In [107]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [108]:
raw_file="Raw data 28-April'25 According to 4_7.5.csv"
gst_file="GST 28-April'25 According to 4_7.5.csv"
tracker_file="EDD Tracker till 28th April'25.csv"
format_file="edd_format.csv"
ops_file="ops_status.csv"
old_tracker_file="old_tracker.csv"


In [109]:
raw_data= pd.read_csv(raw_file,memory_map=True)
gst_data=pd.read_csv(gst_file,memory_map=True)
tracker_data=pd.read_csv(tracker_file,memory_map=True)
master_format=pd.read_csv(format_file,memory_map=True)
ops_status=pd.read_csv(ops_file,memory_map=True)
old_tracker=pd.read_csv(old_tracker_file,memory_map=True)

In [110]:
raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,85331444,OxufPP25967140344013,OxufPP25967140344013,85331444,2024-10-14 10:55:56.000,2024-10-14 10:56:32.000,1.926790e+09,Offline Unorganized,QR,PTY,12585331444,"{""sellerId"":""12585331444""}",PTYES,AHJPB2257J,AHJPB2257J,27AHJPB2257J1ZD,Yes,Agriculture,Fertilizer Dealers,820,PROPRIETORSHIP,2200.0,750000,366.666667,11366.666667,1,1,0.002933,Not_required,Pending for share,100000.0
1,85090959,OzMgfa77912316485832,OzMgfa77912316485832,85090959,2024-09-23 11:39:43.000,2024-09-23 11:40:07.000,2.946200e+08,Petrol Pumps,EDC,PTY,15811530922,"{""sellerId"":""15811530922""}",PTYES,AFQPA9586C,AFQPA9586C,29AFQPA9586C1Z0,Yes,LFR and Brands,Grocery,5411,PROPRIETORSHIP,629230.0,750000,20297.741935,629230.000000,1,1,0.838973,Not_required,Pending for share,700000.0
2,62292787,PALMGR28652901614627,PALMGR28652901614627,62292787,2021-02-26 13:44:59.000,2025-04-18 16:11:49.000,NaN,Enterprise,EDC,PTY,16962292787,"{""sellerId"":""16962292787""}",PTYES,AAECM2690M,AANPC3204B,07AAECM2690M1Z3,Yes,Travel,Hotel,7011,PRIVATE_LIMITED,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [111]:
gst_data=gst_data.rename(columns={'_col3':'updated_at','solution_value.1':'gst'})


# Convert 'date' column to datetime type
gst_data['updated_at'] = pd.to_datetime(gst_data['updated_at'])


gst_data.head(3)

,solution_value,cust_id,gst,updated_at
0,KnJWxD55988386613477,1.402779e+09,Slab: Rs. 40 lakhs to 1.5 Cr.,2025-04-28
1,eAddPc37776209652402,2.840284e+07,Slab: Rs. 5 Cr. to 25 Cr.,2025-04-28
2,SbVsJd95302173729369,1.948164e+09,Slab: Rs. 0 to 40 lakhs,2025-04-28


In [112]:


# Sort by id and date (latest date first)
gst_data = gst_data.sort_values(by=['solution_value', 'updated_at'], ascending=[True, False])

# Drop duplicates: keep the first (which is the latest date because of sort)
gst_data = gst_data.drop_duplicates(subset='solution_value', keep='first')

# Reset index if needed
gst_data = gst_data.reset_index(drop=True)

In [113]:
# Convert the two columns to datetime first
raw_data['mid_creation_date'] = pd.to_datetime(raw_data['mid_creation_date'], errors='coerce')
raw_data['channel_date'] = pd.to_datetime(raw_data['channel_date'], errors='coerce')

# Then format them to 'dd-mm-yyyy' string format
raw_data['mid_creation_date'] = raw_data['mid_creation_date'].dt.strftime('%d-%m-%Y')
raw_data['channel_date'] = raw_data['channel_date'].dt.strftime('%d-%m-%Y')

raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,85331444,OxufPP25967140344013,OxufPP25967140344013,85331444,14-10-2024,14-10-2024,1.926790e+09,Offline Unorganized,QR,PTY,12585331444,"{""sellerId"":""12585331444""}",PTYES,AHJPB2257J,AHJPB2257J,27AHJPB2257J1ZD,Yes,Agriculture,Fertilizer Dealers,820,PROPRIETORSHIP,2200.0,750000,366.666667,11366.666667,1,1,0.002933,Not_required,Pending for share,100000.0
1,85090959,OzMgfa77912316485832,OzMgfa77912316485832,85090959,23-09-2024,23-09-2024,2.946200e+08,Petrol Pumps,EDC,PTY,15811530922,"{""sellerId"":""15811530922""}",PTYES,AFQPA9586C,AFQPA9586C,29AFQPA9586C1Z0,Yes,LFR and Brands,Grocery,5411,PROPRIETORSHIP,629230.0,750000,20297.741935,629230.000000,1,1,0.838973,Not_required,Pending for share,700000.0
2,62292787,PALMGR28652901614627,PALMGR28652901614627,62292787,26-02-2021,18-04-2025,NaN,Enterprise,EDC,PTY,16962292787,"{""sellerId"":""16962292787""}",PTYES,AAECM2690M,AANPC3204B,07AAECM2690M1Z3,Yes,Travel,Hotel,7011,PRIVATE_LIMITED,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [114]:
tracker_data=tracker_data.fillna(0)
tracker_data.isna().sum()

Bank Response    0
Final Remarks    0
dtype: int64

In [115]:
len(ops_status)+len(tracker_data)+len(old_tracker)

451243

In [116]:
tracker_data=tracker_data.rename(columns={'Bank Response':'bank_response','Final Remarks':'final_remarks'})
old_tracker=old_tracker.rename(columns={'bank_response':'bank_response','Final Remarks':'final_remarks'})
ops_status=ops_status.rename(columns={'sellerid':'bank_response','OPS status':'final_remarks'})



ops_status_data= pd.concat([ops_status, tracker_data,old_tracker], axis=0, ignore_index=True)
print('totol tracker data ',len(ops_status_data))
print(ops_status_data.isna().sum())


totol tracker data  451243
bank_response    0
final_remarks    0
dtype: int64


In [117]:
len(raw_data)

319768

In [118]:
# Make sure everything is a string first
raw_data['sellerid'] = raw_data['sellerid'].astype(str)
raw_data['entity_id'] = raw_data['entity_id'].astype(str)
raw_data['parent_entity_id'] = raw_data['parent_entity_id'].astype(str)

# Calculate the length of each entity_id
raw_data['entity_len'] = raw_data['entity_id'].str.len()

# Extract from seller_id the last N characters based on entity_len
raw_data['seller_tail'] = raw_data.apply(lambda x: x['sellerid'][-int(x['entity_len']):], axis=1)


# Step 3: Compare both
raw_data['is_entity_match'] = raw_data['seller_tail'] == raw_data['entity_id']
raw_data['is_parent_match'] = raw_data['seller_tail'] == raw_data['parent_entity_id']

# Step 4: Create final match column
raw_data['final_match'] = raw_data['is_entity_match'] | raw_data['is_parent_match']

raw_data = raw_data[raw_data['final_match']]


len(raw_data)



278952

In [119]:
master_format[['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank']]=raw_data[['parent_id', 'mid','parent_entity_id','entity_id',
       'mid_creation_date', 'channel_date', 'cust_id', 'channel', 'edc_flag',
       'vpa_type', 'sellerid', 'bank_response', 'bank_name', 'pan_no_business',
       'pan_no_personal', 'gstin', 'gst_flag', 'category', 'sub_category',
       'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag', 'drr',
       'month_expected_gmv', 'total_child_mid', 'rank']]

len(master_format)

278952

In [120]:
gst_mapping = gst_data.set_index('solution_value')['gst'].to_dict()
master_format['GST Turn over'] = master_format['parent_id'].map(gst_mapping)

# Fill missing GSTs with 'NA'
master_format['GST Turn over'] = master_format['GST Turn over'].fillna('NA')

In [121]:
ops_status_mapping = ops_status_data.set_index('bank_response')['final_remarks'].to_dict()
master_format['OPS team status'] = master_format['bank_response'].map(ops_status_mapping)

# Fill missing GSTs with 'NA'
master_format['OPS team status'] = master_format['OPS team status'].fillna('Pending for share')

In [122]:
master_format.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1'],
      dtype='object')

In [123]:

master_format['Proposed Limit'] = np.where(
    master_format['Month_expected_GMV'].isna(), 
    master_format['limit_flag'], 
    np.maximum(np.ceil(master_format['Month_expected_GMV'] / 1e5) * 1e5, master_format['limit_flag'])
)


c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in ceil
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [124]:

# First replace NaN in 'gmv' with 0 for calculation
master_format['% usage'] = (master_format['gmv'].fillna(0) / master_format['limit_flag']) * 100




In [125]:

# Make sure % usage is numeric
master_format['% usage'] = pd.to_numeric(master_format['% usage'], errors='coerce')

# Define bins and labels
bins = [-float('inf'), 5, 15, 40, 60, 80, 100, float('inf')]
labels = ['1_<5 %', '2_5 to 15 %', '3_15-40%', '4_40-60%', '5_60-80%', '6_80-100%', '7.above 100%']

# Apply binning
master_format['Grouping'] = pd.cut(master_format['% usage'], bins=bins, labels=labels, right=False)



master_format['Grouping'].value_counts()


1_<5 %          147142
7.above 100%     48342
3_15-40%         30937
2_5 to 15 %      27469
4_40-60%         12052
5_60-80%          8025
6_80-100%         4985
Name: Grouping, dtype: int64

In [126]:

# Suppose your DataFrame is named df
liquor_categories = ['Alcoholic Beverages', 'Restaurants and Bars', 'wine and liquor shop', 'Wine producers','Alcoholic beverages']

master_format['Liquor/Others'] = master_format['sub_category'].apply(lambda x: 'Liquor' if x in liquor_categories else 'Others')


master_format['Liquor/Others'].value_counts()



Others    266703
Liquor     12249
Name: Liquor/Others, dtype: int64

In [127]:


master_format['Limit Required/Not'] = np.where(master_format['Proposed Limit'] > master_format['limit_flag'], 'Required', 'Not_required')

master_format['Limit Required/Not'].value_counts()


Not_required    219626
Required         59326
Name: Limit Required/Not, dtype: int64

In [128]:

master_format['Sum of GMV'] = master_format.groupby('bank_response')['gmv'].transform('sum')

master_format['Sum of DRR'] = master_format.groupby('bank_response')['DRR'].transform('sum')

master_format['Sum of Month Expected GMV'] = master_format.groupby('bank_response')['Month_expected_GMV'].transform('sum')

master_format['Sum of limit_flag'] = master_format.groupby('bank_response')['limit_flag'].transform('sum')

# Use numpy's vectorized operations  (MAX and ROUNDUP equivalent)
master_format['Sum of Proposed Limit'] = np.maximum(np.ceil(master_format['Sum of Month Expected GMV'] / 10**5) * 10**5, master_format['Sum of limit_flag'])

# Use numpy for the IF condition (AO2 > AN2)
master_format['Limit Required/Not.1'] = np.where(master_format['Sum of Proposed Limit'] > master_format['Sum of limit_flag'], "Required", "Not_required")

In [129]:
master_format.head(5)

,parent_id,Child MID,Parent Entity_Id,Child Entity_Id,MID Creation Date,PTY Creation Date,Cust_id,channel,EDC Flag,vpa_type,sellerid,bank_response,bank_name,Pan No Business,Pan No Personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,DRR,Month_expected_GMV,total_Child_mid,rank,Proposed Limit,% usage,Grouping,GST Turn over,Liquor/Others,Limit Required/Not,Business status,OPS team status,YBL status,Sum of GMV,Sum of DRR,Sum of Month Expected GMV,Sum of limit_flag,Sum of Proposed Limit,Limit Required/Not.1
0,OxufPP25967140344013,OxufPP25967140344013,85331444,85331444,14-10-2024,14-10-2024,1.926790e+09,Offline Unorganized,QR,PTY,12585331444,"{""sellerId"":""12585331444""}",PTYES,AHJPB2257J,AHJPB2257J,27AHJPB2257J1ZD,Yes,Agriculture,Fertilizer Dealers,820,PROPRIETORSHIP,2200.0,750000,366.666667,11366.666667,1,1,750000.0,0.293333,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,YBL Rejected pending at business,NaN,2200.0,366.666667,11366.666667,750000,750000.0,Not_required
2,PALMGR28652901614627,PALMGR28652901614627,62292787,62292787,26-02-2021,18-04-2025,NaN,Enterprise,EDC,PTY,16962292787,"{""sellerId"":""16962292787""}",PTYES,AAECM2690M,AANPC3204B,07AAECM2690M1Z3,Yes,Travel,Hotel,7011,PRIVATE_LIMITED,NaN,750000,NaN,NaN,1,1,750000.0,0.000000,1_<5 %,NA,Others,Not_required,NaN,ODD/EDD closed and no action required (Limit a...,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required
3,PBoRUt20745378320983,PBoRUt20745378320983,84611443,84611443,06-08-2024,06-08-2024,2.903697e+08,Offline Unorganized,QR,PTY,12384611443,"{""sellerId"":""12384611443""}",PTYES,AMCPJ8235E,AMCPJ8235E,37AMCPJ8235E1Z5,Yes,Cable and Broadband,D2H,4899,PROPRIETORSHIP,17015.0,750000,4253.750000,131866.250000,1,1,750000.0,2.268667,1_<5 %,NA,Others,Not_required,NaN,Closed,NaN,17015.0,4253.750000,131866.250000,750000,750000.0,Not_required
4,SAHARA78273403427955,SAHARA78273403427955,84155363,84155363,11-06-2024,11-06-2024,1.282206e+09,Petrol Pumps,EDC,PTY,15884155363,"{""sellerId"":""15884155363""}",PTYES,AJIPT5580B,AJIPT5580B,29AJIPT5580B2ZV,Yes,Retail and Shopping,Bags and Luggage,5948,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.000000,1_<5 %,NA,Others,Not_required,NaN,Pending for share,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required
5,SAHUBA11829635808261,SAHUBA11829635808261,86174764,86174764,17-12-2024,23-12-2024,6.526717e+08,Petrol Pumps,EDC,PTY,14986174764,"{""sellerId"":""14986174764""}",PTYES,DTQPS7492E,DTQPS7492E,09DTQPS7492E1ZR,Yes,Retail and Shopping,Electrical Shop,5065,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.000000,1_<5 %,NA,Others,Not_required,NaN,Uploaded,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required


In [131]:
#master_format.to_csv('master_format.csv',index=False)

In [132]:
pending_data =master_format[master_format['OPS team status']=='Pending for share']
len(pending_data)

46847

In [133]:
pending_data['category'].unique()

array(['Retail and Shopping', 'Food', 'Business Services',
       'Automobiles and Vehicles', 'CDIT', 'Individual Services',
       'Agriculture', 'Wholesale Trade and B2B', 'Travel',
       'Entertainment', 'Milk Dairy and Cooperative',
       'Beauty and Wellness', 'Healthcare', 'LFR and Brands',
       'Event Services', 'GROCERY', 'Personal Services', 'Home Services',
       'Gas and Petrol', 'Utilities', 'Education', 'NGO', 'Grocery',
       'Devotion', 'Corporate Alliance', 'Cable and Broadband',
       'Tuition and Classes', 'Logistics', 'Housing', 'Airport',
       'Digital Goods', 'Hotel', 'Transport', 'Tolls',
       'Parking and Tolls', 'Government', 'BFSI', 'Parking'], dtype=object)

In [134]:
# Create a new column 'remark'
pending_data['Remark'] = ''

# Set remark as 'share' where cat is 'Gas and Petrol' or 'Gas Petrol'
pending_data.loc[pending_data['category'].isin(['Gas and Petrol', 'Gas Petrol']), 'Remark'] = 'O&G'

C:\Users\avnish1.yadav\AppData\Local\Temp\ipykernel_14044\2633286974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pending_data['Remark'] = ''


In [135]:

special_cat = {
    'Category': [
        'Retail and Shopping', 'Food', 'Retail and Shopping', 'Wholesale Trade and B2B',
        'Healthcare', 'Healthcare', 'Education', 'Education', 'Education', 'Education',
        'Healthcare', 'Food','Retail and Shopping'
    ],
    'Subcategory': [
        'wine and liquor shop', 'Restaurants and Bars', 'Alcoholic Beverages', 'Wine producers',
        'Hospital', 'Pharmacy', 'School - Secondary & Higher Secondary', 'School - Primary',
        'School', 'Private College', 'Medical College', 'Food','Alcoholic beverages'
    ],
    'Main Group': [
        'Liquor', 'Liquor', 'Liquor', 'Liquor',
        'Hospital', 'Pharmacy', 'Education', 'Education',
        'Education', 'Education', 'Education', 'Food','Liquor'
    ]
}

special_category = pd.DataFrame(special_cat)

In [140]:
pending_data.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1', 'Remark'],
      dtype='object')

In [137]:

# Perform a left merge to get the corresponding Main Group based on cat and sub_cat
pending_data = pd.merge(pending_data, special_category, how='left', left_on=['category', 'sub_category'], right_on=['Category', 'Subcategory'])

# Update the 'remark' column only where there is a match


pending_data['Remark'] = pending_data.apply(
    lambda row: row['Main Group'] if pd.notna(row['Main Group']) else row['Remark'],
    axis=1
)


pending_data = pending_data.drop(['Category', 'Subcategory','Main Group'], axis=1)

In [138]:
pending_data['Remark'].value_counts()

             46560
Liquor         132
O&G            116
Pharmacy        34
Hospital         4
Education        1
Name: Remark, dtype: int64

In [142]:
pending_data['bank_response_count'] = pending_data['bank_response'].map(pending_data['bank_response'].value_counts())

In [146]:
turn_over = {
    'GST Return Slab': [
        'without GSTIN only pan With Business Proof',
        'GSTIN Turnover NA',
        'Slab: Rs. 0 to 40 lakhs',
        'Slab: Rs. 40 lakhs to 1.5 Cr.',
        'Slab: Rs. 1.5 Cr. to 5 Cr.',
        'Slab: Rs. 5 Cr. to 25 Cr.',
        'Slab: Rs. 25 Cr. to 100 Cr.',
        'Slab: Rs. 100 Cr. to 500 Cr.',
        'Slab: Rs. 500 Cr. and above',
        'Slab: Rs. 25 Cr. to 100 Cr.',
        'Slab: Rs.0 to 40 lakhs',
        'Slab: Rs.1.5 Cr. to 5 Cr.',
        'Slab: Rs.25 Cr. to 100 Cr.',
        'Slab: Rs.40 lakhs to 1.5 Cr.',
        'Slab: Rs.5 Cr. to 25 Cr.',
        'NA'
    ],
    'GST Limit': [
        600000,
        750000,
        750000,
        750000,
        2500000,
        6000000,
        10000000,
        10000000,
        10000000,
        10000000,
        750000,
        2500000,
        10000000,
        750000,
        6000000,
        600000
    ]
}

gst_turn_over = pd.DataFrame(turn_over)